In [85]:
import os
import shutil
import pandas as pd
import csv
import psycopg2

In [ ]:
# 1. 파일명에 '사용자정의채널'이 들어간 파일을 읽는다.

# 2. 유효성 검사를 한다.

# 3. 전처리를 한다.

# 4. db에 입력한다.

In [88]:
# 1. 파일명에 '사용자정의채널'이 들어간 파일을 읽는다.

root_path = 'C:/dataHub'
file_list = os.listdir(root_path)

사용자정의채널 = [file for file in file_list if '사용자정의채널' in file]

사용자정의채널

['사용자정의채널_2023-04-23_2023-04-23.xlsx']

In [89]:
# 2. 유효성 검사를 한다.

checked_file = []
check_date_list = ['2023-04-23']

for check_date in check_date_list:
    for file in 사용자정의채널:
        if check_date + '_' + check_date in file:
            checked_file.append(file)

checked_file

['사용자정의채널_2023-04-23_2023-04-23.xlsx']

In [90]:
# 3. 전처리를 한다.

for file in checked_file:
    df = pd.read_excel(root_path + '/' + file)
    df['날짜'] = file[8:18]
    col_order = df.columns.to_list()[-1:] + df.columns.to_list()[0:-1]
    df = df[col_order]
    df.to_csv(root_path + '/dataSub/사용자정의채널/' + file[:-5] + '.csv', encoding='cp949', index=False)
    os.remove(root_path + '/' + file)

c:\Users\나현중\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [112]:
# 4. db에 입력한다.

conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


preprocessed_file = os.listdir(root_path + '/dataSub/사용자정의채널/')
preprocessed_file.remove('DB입력완료')


for file in preprocessed_file:
    df = pd.read_csv(root_path + '/dataSub/사용자정의채널/' + file, encoding='cp949')
    data = df.values.tolist()
    for d in data:
        cursor.execute('INSERT INTO "Naver_Custom_Order" (yymmdd, device, nt_source, nt_medium, nt_detail, nt_keyword, customer_cnt, inflow_cnt, page_cnt, page_inflow_cnt, order_cnt, order_inflow_per, order_price, order_inflow_price, contribute_cnt, contribute_inflow_per, contribute_price, contribute_inflow_price) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', d)


    shutil.move(root_path + '/dataSub/사용자정의채널/' + file, root_path + '/dataSub/사용자정의채널/DB입력완료')

conn.commit()
conn.close()